## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-22-50-11 +0000,nyt,"Shooting at Mormon Church in Grand Blanc, Mich...",https://www.nytimes.com/live/2025/09/28/us/gra...
1,2025-09-28-22-45-30 +0000,nyt,"Trump’s Plan to Send Troops to Portland, Orego...",https://www.nytimes.com/2025/09/28/us/portland...
2,2025-09-28-22-45-03 +0000,nyt,"Venezuela, America and the Specter of Regime C...",https://www.nytimes.com/2025/09/28/world/venez...
3,2025-09-28-22-44-39 +0000,nyt,Forget Labubus: We’re Collecting Semi-Cured Ge...,https://www.nytimes.com/2025/09/28/business/se...
4,2025-09-28-22-40-15 +0000,bbc,Pro-EU party in Moldova set to win election mi...,https://www.bbc.com/news/articles/cx2rdlj8ejgo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,22
2,church,12
0,shooting,12
152,new,10
5,michigan,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
41,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,74
79,2025-09-28-17-44-38 +0000,nypost,Eric Adams rips political ‘extremism’ as he dr...,https://nypost.com/2025/09/28/us-news/eric-ada...,69
61,2025-09-28-19-26-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...,60
25,2025-09-28-21-27-11 +0000,bbc,At least two dead and several injured in shoot...,https://www.bbc.com/news/articles/ceq2vd15glwo...,60
27,2025-09-28-21-22-46 +0000,wapo,"At least two dead, eight injured in Michigan c...",https://www.washingtonpost.com/nation/2025/09/...,59


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
41,74,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
79,69,2025-09-28-17-44-38 +0000,nypost,Eric Adams rips political ‘extremism’ as he dr...,https://nypost.com/2025/09/28/us-news/eric-ada...
25,60,2025-09-28-21-27-11 +0000,bbc,At least two dead and several injured in shoot...,https://www.bbc.com/news/articles/ceq2vd15glwo...
125,37,2025-09-28-13-13-51 +0000,bbc,Building of three new towns will start before ...,https://www.bbc.com/news/articles/cly1geen679o...
53,28,2025-09-28-19-48-52 +0000,nypost,Iraq War veteran Thomas Sanford ID’d as gunman...,https://nypost.com/2025/09/28/us-news/iraq-war...
131,28,2025-09-28-12-12-08 +0000,cbc,"Russia pounds Kyiv, other regions in mass dron...",https://www.cbc.ca/news/world/russia-mass-dron...
172,26,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
98,22,2025-09-28-15-48-56 +0000,nypost,James Comey’s ex-No. 2 gives mealy-mouthed ans...,https://nypost.com/2025/09/28/us-news/comeys-e...
14,20,2025-09-28-21-59-11 +0000,startribune,Three pedestrians struck and killed in Twin Ci...,https://www.startribune.com/man-killed-after-b...
114,20,2025-09-28-14-16-07 +0000,nypost,"UN sets arms embargo, other sanctions again on...",https://nypost.com/2025/09/28/world-news/un-se...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
